<br> </br>
<font size = 8> <center> AI Strategy and Digital transformation </center> </font>
<font size = 6> <center>  <b> 1. Data preparation </b> </center>
<br>
<font size = 5> <center> Piotr Wójcik </center> </font>
<font size = 5> <center> University of Warsaw, Poland
<font size = 5> <center> pwojcik@wne.uw.edu.pl
<br> </br>
<font size = 5> <center>  January 2025 </center> </font>
</center> </font>

In [1]:
# change working directory
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/szkolenia/2025-01_Bucharest'

ModuleNotFoundError: No module named 'google.colab'

In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
import pickle

# importing freqtable() function defined by the lecturer in PW_functions.py
from PW_functions import freqtable

# Description of `churn` dataset

We will show the examples for classification based on the `churn` dataset.

The dataset includes 10127 observations and the following columns:
  
* `customer_id` - unique observation identifier
* `customer_age` - age of the customer in years
* `customer_gender` - gender of the customer
* `customer_number_of_dependents` - number of dependents on the customer
* `customer_education` - education level of the customer
* `customer_civil_status` - civil status of the customer
* `customer_salary_range` - range of the annual salary of the customer
* `customer_relationship_length` - length of customer’s relationship with bank in months
* `customer_available_credit_limit` - available limit on the customer’s credit card account
* `credit_card_classification` - classification of the card (Blue, Silver, Gold, Platinum)
* `total_products` - total number of products held by the customer in the bank
* `period_inactive` - period in the last year when customer was inactive (in months)
* `contacts_in_last_year` - number of contacts with the customer in the last year
* `credit_card_debt_balance` - total card debt balance on the credit card account
* `remaining_credit_limit` - remaining limit on the customer’s credit card account (average in last year)
* `transaction_amount_ratio` - ratio in total amount of transactions in the 4th quarter against the 1st quarter
* `total_transaction_amount` - total amount of transactions in the last year
* `total_transaction_count` - total number of transactions in the last year
* `transaction_count_ratio` - ratio in total count of transactions in the 4th quarter against the 1st quarter
* `average_utilization` - average card utilization (percentage used of total limit)
* `account_status` - customer account status: closed, open (outcome variable, only in the training sample)

Lets import the data and check its structure


In [92]:
# import data
churn = pd.read_csv("data/churn.csv")

churn.head() # Shows the first five rows of the dataset

,customer_id,customer_age,customer_gender,customer_number_of_dependents,customer_education,customer_civil_status,customer_salary_range,customer_relationship_length,customer_available_credit_limit,credit_card_classification,...,period_inactive,contacts_in_last_year,credit_card_debt_balance,remaining_credit_limit,transaction_amount_ratio,total_transaction_amount,total_transaction_count,transaction_count_ratio,average_utilization,account_status
0,755410,38.0,F,2,High School,Married,40-60K,31,1593.0,Blue,...,2,4,1091,502.0,0.870,4136.0,67,0.718,0.685,open
1,568093,46.0,F,2,Graduate,Unknown,below 40K,40,6568.0,Blue,...,2,2,0,6568.0,0.101,1507.0,33,0.222,0.000,closed
2,595389,43.0,M,1,High School,Married,80-120K,30,34516.0,Silver,...,1,3,2045,32471.0,0.590,4081.0,54,0.421,0.059,open
3,287252,46.0,F,4,High School,Married,below 40K,36,2374.0,Blue,...,2,1,1332,1042.0,0.686,4253.0,81,0.884,0.561,open
4,231901,40.0,M,4,High School,Single,80-120K,29,12978.0,Blue,...,3,2,0,12978.0,0.628,14134.0,85,0.700,0.000,open


In [93]:
# let's check the structure of the dataset
churn.info()         # Displays index, column dtypes, non-null counts, etc.
churn.describe()     # Provides descriptive statistics for numerical columns

# customer_id is a unique observation identifier

# there are 6 categorical variables which have to be recoded
# (Dtype = 'object' - lets change it to dtype 'category'
# AFTER we impute potential missing values with a new level).
# Otherwise we would need to take an additional step of first extending a list of levels:
# .cat.add_categories()
# see e.g. here for more details: https://aeturrell.github.io/python4DS/categorical-data.html
#       or here: https://pandas.pydata.org/docs/user_guide/categorical.html

# missing values in:
# - two categorical columns: customer_gender, customer_salary_range
# - two numerical columns: customer_age, total_transaction_amount

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   customer_id                      10127 non-null  int64  
 1   customer_age                     9503 non-null   float64
 2   customer_gender                  9109 non-null   object 
 3   customer_number_of_dependents    10127 non-null  int64  
 4   customer_education               10127 non-null  object 
 5   customer_civil_status            10127 non-null  object 
 6   customer_salary_range            9446 non-null   object 
 7   customer_relationship_length     10127 non-null  int64  
 8   customer_available_credit_limit  10127 non-null  float64
 9   credit_card_classification       10127 non-null  object 
 10  total_products                   10127 non-null  int64  
 11  period_inactive                  10127 non-null  int64  
 12  contacts_in_last_y

,customer_id,customer_age,customer_number_of_dependents,customer_relationship_length,customer_available_credit_limit,total_products,period_inactive,contacts_in_last_year,credit_card_debt_balance,remaining_credit_limit,transaction_amount_ratio,total_transaction_amount,total_transaction_count,transaction_count_ratio,average_utilization
count,10127.000000,9503.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,9720.000000,10127.000000,10127.000000,10127.000000
mean,550508.987854,46.317900,2.346203,35.928409,10036.343784,4.147329,2.341167,2.455317,1162.814061,7469.139637,0.759941,5253.711934,64.858695,0.817544,0.274894
std,261237.656234,8.001227,1.298908,7.986416,17629.707395,3.183477,1.010622,1.106225,814.987335,9090.685324,0.219207,7402.259900,23.472570,0.619906,0.275691
min,100069.000000,26.000000,0.000000,13.000000,1438.300000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,323605.000000,41.000000,1.000000,31.000000,2578.500000,3.000000,2.000000,2.000000,359.000000,1324.500000,0.631000,2196.000000,45.000000,0.588000,0.023000
50%,552548.000000,46.000000,2.000000,36.000000,4696.000000,4.000000,2.000000,2.000000,1276.000000,3474.000000,0.736000,3971.000000,67.000000,0.711000,0.176000
75%,777326.000000,52.000000,3.000000,40.000000,11767.500000,5.000000,3.000000,3.000000,1784.000000,9859.000000,0.859000,4823.000000,81.000000,0.838000,0.503000
max,999911.000000,73.000000,5.000000,56.000000,310644.000000,36.000000,6.000000,6.000000,2517.000000,34516.000000,3.397000,117159.000000,139.000000,16.250000,0.999000


In [94]:
# table of frequencies for "customer_gender" and "customer_salary_range"

# using the Lecturer function to show frequencies
print(freqtable(churn['customer_gender'], dropna = False))

print(freqtable(churn['customer_salary_range'])) # dropna = False is default

                 count    percent
customer_gender                  
F                 4838  47.773279
M                 4271  42.174385
NaN               1018  10.052335
                       count    percent
customer_salary_range                  
below 40K               3327  32.852770
40-60K                  1666  16.451071
80-120K                 1436  14.179915
60-80K                  1302  12.856720
Unknown                 1030  10.170830
120K and more            685   6.764096
NaN                      681   6.724598


In [95]:
# 1. Imputation of categorical predictors

# replace NaN (missing) values in two columns with the string "unknown".
churn['customer_gender'] = churn['customer_gender'].fillna('Unknown')
churn['customer_salary_range'] = churn['customer_salary_range'].fillna('Unknown')

In [96]:
# table of frequencies for "customer_gender" and "customer_salary_range" after imputation
print(freqtable(churn['customer_gender'], dropna = False))
print(freqtable(churn['customer_salary_range'], dropna = False))

                 count    percent
customer_gender                  
F                 4838  47.773279
M                 4271  42.174385
Unknown           1018  10.052335
                       count    percent
customer_salary_range                  
below 40K               3327  32.852770
Unknown                 1711  16.895428
40-60K                  1666  16.451071
80-120K                 1436  14.179915
60-80K                  1302  12.856720
120K and more            685   6.764096


In [97]:
# lets check the values of one of the ordinal predictors

print(freqtable(churn['customer_salary_range'], dropna = False))

                       count    percent
customer_salary_range                  
below 40K               3327  32.852770
Unknown                 1711  16.895428
40-60K                  1666  16.451071
80-120K                 1436  14.179915
60-80K                  1302  12.856720
120K and more            685   6.764096


In [98]:
# Lets convert the categorical columns to 'category' type

# List of categorical (nominal) columns
categorical_columns = ['customer_gender',
                       'customer_civil_status',
                       'credit_card_classification',
                       'account_status']

churn[categorical_columns] = churn[categorical_columns].astype('category')

# However, two of them: customer_education and salary_range are ordinal (have interal order)

# lets take this into account - categorical data type has a categories= and a ordered= property

# For ordered categorical data certain operations can be applied:
#  you can sort values (with .sort_values), and apply .min and .max

# Define the order of salary ranges - lets put Unknown first (it will be later drooped)
salary_order = ['Unknown', 'below 40K', '40-60K', '60-80K', '80-120K', '120K and more']

# Convert the 'customer_salary_range' column to an ordinal categorical type
churn['customer_salary_range'] = pd.Categorical(churn['customer_salary_range'],
                                                categories = salary_order,
                                                ordered = True)

# here Unknown also put at the first place
education_order = ['Unknown', 'Uneducated', 'High School', 'College', 'Graduate', 'Post-Graduate', 'Doctorate']

churn['customer_education'] = pd.Categorical(churn['customer_education'],
                                                categories = education_order,
                                                ordered = True)

print("Minimum value of customer salary range: ", churn['customer_salary_range'].min())

print("Maximum value of customer education: ", churn['customer_education'].max())

Minimum value of customer salary range:  Unknown
Maximum value of customer education:  Doctorate


In [99]:
# 2. imputation of numerical predictors

# Train Test Split has to be done BEFORE imputation which takes into account the distribution of variables

# random split into train (70%)/test (30%) stratified by 'account_status'
churn_train, churn_test = train_test_split(
    churn,
    test_size = 0.3,
    stratify = churn['account_status'],
    random_state = 123 # keep constant for reproducibility
)

# Lets check the distribution of account_status in train and test samples

print("Training sample")
print(freqtable(churn_train['account_status'], dropna = False))
print("Test sample")
print(freqtable(churn_test['account_status'], dropna = False))

Training sample
                count    percent
account_status                  
open             5949  83.930587
closed           1139  16.069413
Test sample
                count    percent
account_status                  
open             2551  83.942086
closed            488  16.057914


In [100]:
# KNNimputation (k = 5) based on numerical columns (EXCEPT from the customer_id)

# Identify numeric columns in the training data (the same are in test)
churn_numeric_cols = churn_train.select_dtypes(include = [np.number]).columns
# and remove the first element (customer_id)
churn_numeric_cols = churn_numeric_cols[1:]

print(churn_numeric_cols)

Index(['customer_age', 'customer_number_of_dependents',
       'customer_relationship_length', 'customer_available_credit_limit',
       'total_products', 'period_inactive', 'contacts_in_last_year',
       'credit_card_debt_balance', 'remaining_credit_limit',
       'transaction_amount_ratio', 'total_transaction_amount',
       'total_transaction_count', 'transaction_count_ratio',
       'average_utilization'],
      dtype='object')


In [101]:
# Scale numeric features to z-scores (fit only on training data, but apply on both)
scaler = StandardScaler()

# lets make a copy of each data for scaling
churn_train_scaled = churn_train.copy()
churn_test_scaled = churn_test.copy()

# fit_transform() method applied on training data and transform() method on test data
churn_train_scaled[churn_numeric_cols] = scaler.fit_transform(churn_train[churn_numeric_cols])
churn_test_scaled[churn_numeric_cols] = scaler.transform(churn_test[churn_numeric_cols])
# Using the transform method we apply the same mean and variance calculated from the training data to transform the test data.

# check for more detailed explanation of the difference between .fit_transform() and .transform()
# e.g. here https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe

In [102]:
# KNN imputation (again fit only on the scaled training set and apply on both)
imputer = KNNImputer(n_neighbors = 5)

churn_train_imputed = churn_train_scaled.copy()
churn_test_imputed = churn_test_scaled.copy()

# Fit on train, then transform
churn_train_imputed[churn_numeric_cols] = imputer.fit_transform(
    churn_train_scaled[churn_numeric_cols])

# Transform test - based on the neighbors from the TRAIN sample !!
churn_test_imputed[churn_numeric_cols] = imputer.transform(
    churn_test_scaled[churn_numeric_cols])

In [103]:
# lets check the imputed data
churn_train_imputed.info()
churn_test_imputed.info()

# no more missings
# in addition all numeric columns (imputed via KNN) and are scaled to z-scores

<class 'pandas.core.frame.DataFrame'>
Index: 7088 entries, 10007 to 2264
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   customer_id                      7088 non-null   int64   
 1   customer_age                     7088 non-null   float64 
 2   customer_gender                  7088 non-null   category
 3   customer_number_of_dependents    7088 non-null   float64 
 4   customer_education               7088 non-null   category
 5   customer_civil_status            7088 non-null   category
 6   customer_salary_range            7088 non-null   category
 7   customer_relationship_length     7088 non-null   float64 
 8   customer_available_credit_limit  7088 non-null   float64 
 9   credit_card_classification       7088 non-null   category
 10  total_products                   7088 non-null   float64 
 11  period_inactive                  7088 non-null   float64 
 12  contact

# Encoding of categorial predictors

Most of machine learning algorithms like Linear Regression, Neural Networks, SVMs, KNN, etc., aren’t designed to process text-based categorical data directly. So, it’s crucial to transform categorical data into a numerical form, a process known as data encoding.

Notably, some tree-based ML algorithms like Decision Trees and Random Forests can handle categorical data natively, circumventing the need for encoding. However, for most other algorithms, encoding is a vital preprocessing step.

## One-hot encoding of the **nominal** predictors

In [104]:
# lets apply one-hot-encoding on categorical predictors
# for more details check e.g. here https://www.datacamp.com/tutorial/one-hot-encoding-python-tutorial

# Identify categorical columns in the training data (the same are in the test dataset)
categorical_cols = churn_train_imputed.select_dtypes(include = "category").columns

print(categorical_cols)

Index(['customer_gender', 'customer_education', 'customer_civil_status',
       'customer_salary_range', 'credit_card_classification',
       'account_status'],
      dtype='object')


However, some of the columns are ORDINAL.
One-hot encoding is appropriate when the categories do not have an intrinsic ordering or relationship with each other. This is because one-hot encoding treats each category as a separate entity with no relation to the other categories. One-hot encoding is also useful when the number of categories is relatively small, as the number of columns can become unwieldy for very large numbers of categories.

In [105]:
# one-hot-encoding using the pandas get_dummies()

# lets apply it to NOMINAL columns only
nominal_columns = ['customer_gender', 'customer_civil_status', 'credit_card_classification', 'account_status']

# train
churn_train_encoded = pd.get_dummies(churn_train_imputed,
                                     columns = nominal_columns,
                                     drop_first = True, # we do not need all levels
                                     dtype = int) # bool by default

# test
churn_test_encoded = pd.get_dummies(churn_test_imputed,
                                    columns = nominal_columns,
                                    drop_first = True,
                                    dtype = int)

churn_test_encoded.head()

# now all the recoded categorical predictors are at the end of the dataframe

,customer_id,customer_age,customer_number_of_dependents,customer_education,customer_salary_range,customer_relationship_length,customer_available_credit_limit,total_products,period_inactive,contacts_in_last_year,...,average_utilization,customer_gender_M,customer_gender_Unknown,customer_civil_status_Married,customer_civil_status_Single,customer_civil_status_Unknown,credit_card_classification_Gold,credit_card_classification_Platinum,credit_card_classification_Silver,account_status_open
2089,122823,-1.179263,0.490392,Graduate,below 40K,-1.875667,-0.460621,-0.047547,-0.341229,-0.402278,...,0.178986,0,0,0,1,0,0,0,0,0
8911,674482,0.824459,-0.276069,High School,Unknown,1.382108,-0.146121,-0.363695,-1.324011,-1.306320,...,-0.365464,0,0,1,0,0,0,0,0,1
8411,529000,0.824459,0.490392,Graduate,60-80K,0.880912,-0.094396,0.268601,0.641554,0.501764,...,-0.347316,1,0,0,1,0,0,0,0,1
7311,344732,-0.427867,0.490392,Graduate,Unknown,-0.372079,-0.429807,-0.363695,0.641554,-1.306320,...,1.826854,0,0,0,1,0,0,0,0,1
9211,957784,0.699226,-1.042530,High School,60-80K,1.131510,0.098896,-0.995992,2.607119,-1.306320,...,-0.223907,1,0,0,1,0,0,0,0,1


In [106]:
# for the dependent (last) variable the 'open' status was kept,
# while we prefer to model the 'closed' status

# lets change it manually
# Create a new column 'account_status_closed'
churn_train_encoded['account_status_closed'] = 1 - churn_train_encoded['account_status_open']
churn_test_encoded['account_status_closed'] = 1 - churn_test_encoded['account_status_open']

# Drop the old 'account_status_open' column
churn_train_encoded.drop(columns = ['account_status_open'], inplace = True)
churn_test_encoded.drop(columns = ['account_status_open'], inplace = True)

`pandas.get_dummies` is straightforward to use and it automatically converts only the listed column(s), keeping all the others untouched, but copying them into a resulting dataset.

Another approach is to use `OneHotEncoder` from the sklearn library, which is useful in machine learning tasks.

The primary difference is `pandas.get_dummies` cannot learn encodings - store the rules to apply it on another dataset; it can only perform one-hot-encoding on the dataset you pass as an input.

`sklearn.OneHotEncoder` is a class that can be saved and used to transform other incoming datasets in the future.

However, it requires more data transformations if applied out of the model pipeline.

## Ordinal encoding for the **ordinal** predictors

Ordinal encoding is appropriate when the categories have a natural ordering or relationship with each other, such as in the case of ordinal variables like "small," "medium," and "large." In these cases, the integer values assigned to the categories should reflect the ordering of the categories.

In [107]:
# Initialize the OrdinalEncoder

# it can only encode one column at a time, so we have to define two separate encoders for each individual column

encoder_educ = OrdinalEncoder(categories = [education_order]) # we stored this order before
encoder_salary = OrdinalEncoder(categories = [salary_order])
# one can also add the optional arguments: handle_unknown = 'use_encoded_value', unknown_value = -1

# apply fit and transform on the training data
churn_train_encoded['customer_education'] = encoder_educ.fit_transform(churn_train_encoded[['customer_education']])
# OrdinalEncoder expects the input to be a 2D array, hence the columns are passed as [['column_name']] (a 2D structure),
# rather than ['column_name'] (a 1D structure).
churn_train_encoded['customer_salary_range'] = encoder_salary.fit_transform(churn_train_encoded[['customer_salary_range']])

# and transform also the test data
churn_test_encoded['customer_education'] = encoder_educ.transform(churn_test_encoded[['customer_education']])
churn_test_encoded['customer_salary_range'] = encoder_salary.transform(churn_test_encoded[['customer_salary_range']])

In [108]:
# check if the order learned by the OrdinalEncoder is correct

# Check the label mapping for 'customer_education'
education_mapping = dict(zip(education_order,
                             range(len(education_order))))
# The zip() function pairs elements from education_order and the second sequence provided here
print("Mapping for 'customer_education':")
print(education_mapping)

# Check the label mapping for 'customer_salary_range'
salary_mapping = dict(zip(salary_order,
                          range(len(salary_order))))
print("\nMapping for 'customer_salary_range':")
print(salary_mapping)

# seems to be OK

Mapping for 'customer_education':
{'Unknown': 0, 'Uneducated': 1, 'High School': 2, 'College': 3, 'Graduate': 4, 'Post-Graduate': 5, 'Doctorate': 6}

Mapping for 'customer_salary_range':
{'Unknown': 0, 'below 40K': 1, '40-60K': 2, '60-80K': 3, '80-120K': 4, '120K and more': 5}


In [109]:
churn_test_encoded.info()

# now all the columns are numeric

<class 'pandas.core.frame.DataFrame'>
Index: 3039 entries, 2089 to 9981
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          3039 non-null   int64  
 1   customer_age                         3039 non-null   float64
 2   customer_number_of_dependents        3039 non-null   float64
 3   customer_education                   3039 non-null   float64
 4   customer_salary_range                3039 non-null   float64
 5   customer_relationship_length         3039 non-null   float64
 6   customer_available_credit_limit      3039 non-null   float64
 7   total_products                       3039 non-null   float64
 8   period_inactive                      3039 non-null   float64
 9   contacts_in_last_year                3039 non-null   float64
 10  credit_card_debt_balance             3039 non-null   float64
 11  remaining_credit_limit          

In [115]:
# Let's save the final datasets for further steps
churn_train_encoded.to_csv("outputs/churn_train_prepared.csv",
                           sep = ',',
                           encoding = 'utf-8',
                           index = False,
                           header = True)

churn_test_encoded.to_csv("outputs/churn_test_prepared.csv",
                          sep = ',',
                          encoding = 'utf-8',
                          index = False,
                          header = True)

In [116]:
# Let's save the final datasets for further steps into pickle files
# 'wb' stands for write in a binary mode

with open("outputs/churn_prepared.pkl", "wb") as f:
    pickle.dump(churn_train_encoded, f)
    pickle.dump(churn_test_encoded, f)